In [1]:
#pip install ipywidgets matplotlib
#pip install dash plotly numpy
#pip install

# Main idea (input boxes in python)

In [2]:
'''import numpy as np
import matplotlib.pyplot as plt

def calculate_loan_repayment(loan_amount, annual_interest_rate, loan_term_years, living_costs, inflation_rate, salary_increase_rate, no_repayment_period):
    monthly_interest_rate = annual_interest_rate / 12 / 100
    num_payments = loan_term_years * 12
    monthly_payment = (loan_amount * monthly_interest_rate) / (1 - (1 + monthly_interest_rate) ** -num_payments)

    months = np.arange(num_payments)
    balance = loan_amount
    balances = []

    for month in months:
        if month < no_repayment_period:
            balance = balance * (1 + monthly_interest_rate)
        else:
            balance = balance * (1 + monthly_interest_rate) - monthly_payment
        balances.append(balance)

    fig, ax = plt.subplots()
    ax.plot(months, balances, label='Remaining Balance')
    ax.set_xlabel('Months')
    ax.set_ylabel('Remaining Balance ($)')
    ax.set_title('Loan Repayment Schedule')
    ax.legend()
    plt.show()

    return monthly_payment, balances

# Inputs
loan_amount = float(input("Enter loan amount: "))
annual_interest_rate = float(input("Enter annual interest rate (%): "))
loan_term_years = int(input("Enter loan term (years): "))
monthly_income = float(input("Enter monthly income: "))
living_costs = float(input("Enter monthly living costs: "))
inflation_rate = float(input("Enter annual inflation rate (%): "))
salary_increase_rate = float(input("Enter annual salary increase rate (%): "))
no_repayment_period = int(input("Enter number of months without repayment: "))

# Calculate and display results
monthly_payment, balances = calculate_loan_repayment(
    loan_amount, annual_interest_rate, loan_term_years, living_costs, 
    inflation_rate, salary_increase_rate, no_repayment_period
)

print(f"Monthly Payment: ${monthly_payment:.2f}")
'''

'import numpy as np\nimport matplotlib.pyplot as plt\n\ndef calculate_loan_repayment(loan_amount, annual_interest_rate, loan_term_years, living_costs, inflation_rate, salary_increase_rate, no_repayment_period):\n    monthly_interest_rate = annual_interest_rate / 12 / 100\n    num_payments = loan_term_years * 12\n    monthly_payment = (loan_amount * monthly_interest_rate) / (1 - (1 + monthly_interest_rate) ** -num_payments)\n\n    months = np.arange(num_payments)\n    balance = loan_amount\n    balances = []\n\n    for month in months:\n        if month < no_repayment_period:\n            balance = balance * (1 + monthly_interest_rate)\n        else:\n            balance = balance * (1 + monthly_interest_rate) - monthly_payment\n        balances.append(balance)\n\n    fig, ax = plt.subplots()\n    ax.plot(months, balances, label=\'Remaining Balance\')\n    ax.set_xlabel(\'Months\')\n    ax.set_ylabel(\'Remaining Balance ($)\')\n    ax.set_title(\'Loan Repayment Schedule\')\n    ax.legen

In [3]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
import numpy as np
import functools

# Initialize the Dash app
app = dash.Dash(__name__)

def create_tooltip(text, tooltip_id):
    return html.Span([
        html.Span("?",
                  className="tooltiptext",
                  id=tooltip_id),
        html.Div(text, className="tooltip", id=f"{tooltip_id}-content")
    ], className="tooltip-container")

app.layout = html.Div([
    html.H1("Debt Calculator"),
    
    html.Div([
        html.Label(["Salary or income ($/€/£ per month):", create_tooltip("This is the total amount of money you earn or receive each month.", "salary_tooltip")]),
        dcc.Input(id='salary_input', type='number', value=3000, step=1)
    ], style={'marginBottom': 20, 'marginTop': 20}),
    
    html.Div([
        html.Label(["Loan Amount ($/€/£):", create_tooltip("This is the total amount of money you borrowed.", "loan_tooltip")]),
        dcc.Input(id='loan_amount_input', type='number', value=20000, step=1)
    ], style={'marginBottom': 20, 'marginTop': 20}),
    
    html.Div([
        html.Label(["Annual Interest Rate (%):", create_tooltip("This is the yearly interest rate charged on your loan. (Percentage that adds up on your loan every year)", "interest_tooltip")]),
        dcc.Slider(id='interest_rate_slider', min=0.1, max=15, step=0.1, value=5,
                   marks={i: f'{i}%' for i in range(1, 16)})
    ], style={'marginBottom': 20, 'marginTop': 20}),
    
    html.Div([
        html.Label(["Choose Payment Method:", create_tooltip("Select whether you want to calculate the monthly payment based on the loan term or enter a desired monthly payment.", "payment_method_tooltip")]),
        dcc.RadioItems(
            id='payment_method',
            options=[
                {'label': 'Calculate Monthly Payment (Enter Loan Term)', 'value': 'calculate'},
                {'label': 'Enter Desired Monthly Payment', 'value': 'enter'}
            ],
            value='calculate',
            labelStyle={'display': 'inline-block'}
        )
    ], style={'marginBottom': 20, 'marginTop': 20}),
    
    html.Div(id='loan_term_input_container', children=[
        html.Label(["Loan Term (years):", create_tooltip("This is the number of years you have to repay the loan.", "loan_term_tooltip")]),
        dcc.Input(id='loan_term_input', type='number', value=10, step=1)
    ], style={'marginBottom': 20, 'marginTop': 20}),
    
    html.Div(id='monthly_payment_input_container', children=[
        html.Label(["Desired Monthly Payment ($/€/£):", create_tooltip("This is the amount you want to pay each month to repay the loan.", "monthly_payment_tooltip")]),
        dcc.Input(id='monthly_payment_input', type='number', value=None, step=1)
    ], style={'marginBottom': 20, 'marginTop': 20}),
    
    html.Div([
        html.Label(["Living Costs:", create_tooltip("These are your regular expenses.", "living_costs_tooltip")])
    ], style={'marginBottom': 20, 'marginTop': 20}),
    
    html.Div([
        html.Label(["Rent ($/€/£):", create_tooltip("This is the amount you pay for rent each month.", "rent_tooltip")]),
        dcc.Slider(id='rent_slider', min=0, max=5000, step=1, value=1000,
                   marks={i: f'{i}' for i in range(0, 5001, 500)})
    ], style={'marginBottom': 20, 'marginTop': 20}),
    
    html.Div([
        html.Label(["Groceries ($/€/£):", create_tooltip("This is the amount you spend on groceries each month.", "groceries_tooltip")]),
        dcc.Slider(id='groceries_slider', min=0, max=2000, step=1, value=470,
                   marks={i: f'{i}' for i in range(0, 2001, 200)})
    ], style={'marginBottom': 20, 'marginTop': 20}),
    
    html.Div([
        html.Label(["Travel/Leisure ($/€/£):", create_tooltip("This is the amount you spend on travel and leisure each month.", "travel_tooltip")]),
        dcc.Slider(id='travel_slider', min=0, max=2000, step=50, value=300,
                   marks={i: f'{i}' for i in range(0, 2001, 200)})
    ], style={'marginBottom': 20, 'marginTop': 20}),
    
    html.Div([
        html.Label(["Car Insurance ($/€/£):", create_tooltip("This is the amount you pay for car insurance each month.", "car_insurance_tooltip")]),
        dcc.Input(id='car_insurance_input', type='number', value=150, step=10),
        dcc.Checklist(id='car_insurance_na', options=[{'label': 'Not Applicable', 'value': 'NA'}])
    ], style={'marginBottom': 20, 'marginTop': 20}),
    
    html.Div([
        html.Label(["Health Insurance ($/€/£):", create_tooltip("This is the amount you pay for health insurance each month.", "health_insurance_tooltip")]),
        dcc.Input(id='health_insurance_input', type='number', value=400, step=10),
        dcc.Checklist(id='health_insurance_na', options=[{'label': 'Not Applicable', 'value': 'NA'}])
    ], style={'marginBottom': 20, 'marginTop': 20}),
    
    html.Div([
        html.Label(["Annual Inflation Rate (%):", create_tooltip("This is the rate at which prices (in the general economy)increase each year. The average is 2.54% since 2000", "inflation_tooltip")]),
        dcc.Slider(id='inflation_rate_slider', min=0, max=10, step=0.1, value=2.5,
                   marks={i: f'{i}%' for i in range(0, 11)})
    ], style={'marginBottom': 20, 'marginTop': 20}),
    
    html.Div([
        html.Label(["Annual Salary Increase Rate (%):", create_tooltip("This is the rate at which your salary increases each year. Around 7% (in the United States)", "salary_increase_tooltip")]),
        dcc.Slider(id='salary_increase_rate_slider', min=0, max=10, step=0.1, value=5,
                   marks={i: f'{i}%' for i in range(0, 11)})
    ], style={'marginBottom': 20, 'marginTop': 20}),
    
    html.Div([
        html.Label(["Months Without Repayment:", create_tooltip("This is the number of months you don't have (or want) to make loan repayments.", "no_repayment_tooltip")]),
        dcc.Slider(id='no_repayment_period_slider', min=0, max=24, step=1, value=12,
                   marks={i: f'{i}' for i in range(0, 25, 3)})
    ], style={'marginBottom': 20, 'marginTop': 20}),
    
    html.Div(id='monthly_payment_display', style={'marginBottom': 20, 'marginTop': 20}),
    html.Div(id='loan_term_display', style={'marginBottom': 20, 'marginTop': 20}),
    
    html.Div([
        dcc.Graph(id='loan_repayment_graph')
    ], style={'marginBottom': 20, 'marginTop': 20, 'marginLeft': 20, 'marginRight': 20}),
    
    html.Div([
        dcc.Graph(id='expenses_histogram')
    ], style={'marginBottom': 20, 'marginTop': 20, 'marginLeft': 20, 'marginRight': 20}),
    
    html.Div([
        dcc.Graph(id='interest_pie_chart')
    ], style={'marginBottom': 20, 'marginTop': 20, 'marginLeft': 20, 'marginRight': 20})
])

# Show or hide loan term or monthly payment input based on payment method
@app.callback(
    [Output('loan_term_input_container', 'style'),
     Output('monthly_payment_input_container', 'style')],
    [Input('payment_method', 'value')]
)
def toggle_payment_method_inputs(payment_method):
    if payment_method == 'calculate':
        return {'display': 'block'}, {'display': 'none'}
    else:
        return {'display': 'none'}, {'display': 'block'}

@functools.lru_cache(maxsize=128)
def calculate_monthly_payment(loan_amount, annual_interest_rate, loan_term_years, no_repayment_period):
    if loan_term_years == 0:
        return 0, 0
    monthly_interest_rate = annual_interest_rate / 12 / 100
    num_payments = loan_term_years * 12
    if num_payments == no_repayment_period:
        return 0, num_payments  # Avoid division by zero
    
    if no_repayment_period > 0:
        adjusted_loan_amount = loan_amount * (1 + monthly_interest_rate) ** no_repayment_period
        monthly_payment = (adjusted_loan_amount * monthly_interest_rate) / (1 - (1 + monthly_interest_rate) ** -(num_payments - no_repayment_period))
    else:
        monthly_payment = (loan_amount * monthly_interest_rate) / (1 - (1 + monthly_interest_rate) ** -num_payments)
    return monthly_payment, num_payments

@functools.lru_cache(maxsize=128)
def calculate_loan_term(loan_amount, monthly_payment, annual_interest_rate, no_repayment_period):
    if not monthly_payment or monthly_payment <= 0:
        return 0, 0, 0, 0

    monthly_interest_rate = annual_interest_rate / 12 / 100
    balance = loan_amount * (1 + monthly_interest_rate) ** no_repayment_period
    num_payments = no_repayment_period
    total_interest_paid = 0
    while balance > 0:
        interest_for_month = balance * monthly_interest_rate
        total_interest_paid += interest_for_month
        balance = balance + interest_for_month - monthly_payment
        num_payments += 1
        if num_payments > 600:  # Break loop if payments exceed 50 years to avoid infinite loop
            break
    loan_term_years = num_payments // 12
    loan_term_months = num_payments % 12
    return loan_term_years, loan_term_months, total_interest_paid, num_payments

@functools.lru_cache(maxsize=128)
def calculate_balances(loan_amount, monthly_payment, annual_interest_rate, no_repayment_period, num_payments):
    monthly_interest_rate = annual_interest_rate / 12 / 100
    balance = loan_amount
    balances = []
    total_interest_paid = 0

    for month in range(num_payments):
        if month < no_repayment_period:
            balance = balance * (1 + monthly_interest_rate)
        else:
            interest_for_month = balance * monthly_interest_rate
            total_interest_paid += interest_for_month
            balance = balance + interest_for_month - monthly_payment
        balances.append(balance)
    
    return balances, total_interest_paid

# Define the callback to update the graph and monthly payment
@app.callback(
    [Output('loan_repayment_graph', 'figure'),
     Output('monthly_payment_display', 'children'),
     Output('loan_term_display', 'children'),
     Output('expenses_histogram', 'figure'),
     Output('interest_pie_chart', 'figure')],
    [Input('salary_input', 'value'),
     Input('loan_amount_input', 'value'),
     Input('interest_rate_slider', 'value'),
     Input('payment_method', 'value'),
     Input('loan_term_input', 'value'),
     Input('monthly_payment_input', 'value'),
     Input('rent_slider', 'value'),
     Input('groceries_slider', 'value'),
     Input('travel_slider', 'value'),
     Input('car_insurance_input', 'value'),
     Input('car_insurance_na', 'value'),
     Input('health_insurance_input', 'value'),
     Input('health_insurance_na', 'value'),
     Input('inflation_rate_slider', 'value'),
     Input('salary_increase_rate_slider', 'value'),
     Input('no_repayment_period_slider', 'value')]
)
def update_graph(salary, loan_amount, annual_interest_rate, payment_method, loan_term_years, monthly_payment, rent, groceries, travel, car_insurance, car_insurance_na, health_insurance, health_insurance_na, inflation_rate, salary_increase_rate, no_repayment_period):
    try:
        # Check if car insurance or health insurance are not applicable
        if car_insurance_na and 'NA' in car_insurance_na:
            car_insurance = 0
        if health_insurance_na and 'NA' in health_insurance_na:
            health_insurance = 0

        # Ensure none of the inputs are None or invalid
        rent = rent or 0
        groceries = groceries or 0
        travel = travel or 0
        car_insurance = car_insurance or 0
        health_insurance = health_insurance or 0
        loan_term_years = loan_term_years or 1  # default to 1 year if invalid
        if monthly_payment is None:
            monthly_payment = 0

        living_costs = rent + groceries + travel + car_insurance + health_insurance

        if payment_method == 'calculate':
            if loan_term_years == 0:
                monthly_payment = 0
                num_payments = 0
                loan_term_months = 0
            else:
                monthly_payment, num_payments = calculate_monthly_payment(loan_amount, annual_interest_rate, loan_term_years, no_repayment_period)
                loan_term_months = 0  # No additional months if term is given directly
        else:
            if monthly_payment == 0:
                loan_term_years = 0
                loan_term_months = 0
                total_interest_paid = 0
                num_payments = 0
            else:
                loan_term_years, loan_term_months, total_interest_paid, num_payments = calculate_loan_term(loan_amount, monthly_payment, annual_interest_rate, no_repayment_period)

        if payment_method == 'enter' and monthly_payment > 0 and loan_term_years == 0 and loan_term_months == 0:
            return (go.Figure(), 
                    html.Div("You cannot afford this monthly payment.", style={'fontWeight': 'bold', 'fontSize': '20px', 'color': 'red'}),
                    html.Div("Loan Term: N/A", style={'fontWeight': 'bold', 'fontSize': '20px'}),
                    go.Figure(), 
                    go.Figure())

        balances, total_interest_paid = calculate_balances(loan_amount, monthly_payment, annual_interest_rate, no_repayment_period, num_payments)

        if len(balances) == 0:
            months = np.array([0])
            balances = [0]
        else:
            months = np.arange(num_payments)
        
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=months, y=balances, mode='lines', name='Remaining Balance'))
        fig.update_layout(title='Loan Repayment Schedule',
                          xaxis_title='Months',
                          yaxis_title='Remaining Balance ($/€/£)')
        
        if len(months) > 0:
            fig.update_xaxes(range=[0, max(months)])
            fig.add_shape(type="line",
                          x0=max(months),
                          y0=0,
                          x1=max(months),
                          y1=balances[-1],
                          line=dict(color="Red", dash="dash"))

        if monthly_payment > 0 and salary - living_costs - monthly_payment < 0:
            monthly_payment_display = html.Div(f"Monthly Payment: ${monthly_payment:.2f} (You cannot afford this payment with your current income and expenses.)", style={'fontWeight': 'bold', 'fontSize': '20px', 'color': 'red'})
        else:
            monthly_payment_display = html.Div(f"Monthly Payment: ${monthly_payment:.2f}", style={'fontWeight': 'bold', 'fontSize': '20px'})
        
        loan_term_display = html.Div(f"Loan Term: {loan_term_years} years and {loan_term_months} months", style={'fontWeight': 'bold', 'fontSize': '20px'})

        # Calculate yearly expenses adjusted for inflation
        years = np.arange(1, loan_term_years + 1)
        yearly_expenses = {
            'Rent': [rent * 12 * (1 + inflation_rate / 100) ** (year - 1) for year in years],
            'Groceries': [groceries * 12 * (1 + inflation_rate / 100) ** (year - 1) for year in years],
            'Travel/Leisure': [travel * 12 * (1 + inflation_rate / 100) ** (year - 1) for year in years],
            'Car Insurance': [car_insurance * 12 * (1 + inflation_rate / 100) ** (year - 1) for year in years],
            'Health Insurance': [health_insurance * 12 * (1 + inflation_rate / 100) ** (year - 1) for year in years],
            'Loan Repayment': [monthly_payment * 12 for _ in years]
        }

        # Calculate remaining salary after expenses, adjusted for salary increase
        yearly_salary = [salary * 12 * (1 + salary_increase_rate / 100) ** (year - 1) for year in years]
        remaining_salary = [yearly_salary[year - 1] - sum(expense[year - 1] for expense in yearly_expenses.values()) for year in years]

        # Create traces for the histogram
        expense_traces = [
            go.Bar(name=label, x=years, y=values)
            for label, values in yearly_expenses.items()
        ]
        salary_trace = go.Bar(name='Remaining Salary', x=years, y=remaining_salary)

        expense_histogram = go.Figure(data=expense_traces + [salary_trace])
        expense_histogram.update_layout(barmode='stack', title='Yearly Expenses Breakdown (with inflation)',
                                        xaxis_title='Years', yaxis_title='Amount ($/€/£)')

        # Create a pie chart for interest and principal
        total_principal_paid = loan_amount
        fig_pie = go.Figure(data=[go.Pie(labels=['Principal', 'Interest'],
                                         values=[total_principal_paid, total_interest_paid],
                                         hole=.3)])
        fig_pie.update_layout(title='Proportion of Principal and Interest Paid')
        
        return fig, monthly_payment_display, loan_term_display, expense_histogram, fig_pie

    except Exception as e:
        print(f"Error: {e}")
        # Return empty figures and default text in case of error
        return (go.Figure(), 
                html.Div("Monthly Payment: N/A", style={'fontWeight': 'bold', 'fontSize': '20px'}),
                html.Div("Loan Term: N/A", style={'fontWeight': 'bold', 'fontSize': '20px'}),
                go.Figure(), 
                go.Figure())

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

# Additional CSS for tooltips
app.index_string = '''
<!DOCTYPE html>
<html>
    <head>
        <title>Debt Calculator</title>
        <style>
            .tooltip-container {
                position: relative;
                display: inline-block;
            }
            .tooltip {
                display: none;
                position: absolute;
                background-color: white;
                border: 1px solid black;
                padding: 5px;
                z-index: 10;
                width: 200px;
                bottom: 125%;
                left: 50%;
                margin-left: -100px;
            }
            .tooltip-container:hover .tooltip {
                display: block;
            }
            .tooltip-container .tooltiptext {
                color: white;
                background-color: blue;
                border-radius: 50%;
                padding: 5px;
                cursor: pointer;
                display: inline-block;
                width: 20px;
                height: 20px;
                text-align: center;
                line-height: 20px;
            }
        </style>
    </head>
    <body>
        <div id="react-entry-point">
            {%app_entry%}
        </div>
        <footer>
            {%config%}
            {%scripts%}
            {%renderer%}
        </footer>
    </body>
</html>
'''


http://127.0.0.1:8050/